<a href="https://colab.research.google.com/github/ShamaSharma/SVD/blob/main/Unixcoder_on_Devign(short)_from_paper_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Fix Current Code and Get Baseline Results
!pip install transformers datasets evaluate
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import numpy as np
import evaluate
from sklearn.metrics import confusion_matrix
from google.colab import files

# Load your data (same as before)
print("Upload devign files")
uploaded = files.upload()

dataset = load_dataset("json", data_files={
    "train": "devign_0-512_train.json",
    "validation": "devign_0-512_validate.json",
    "test": "devign_0-512_test.json"
})

# CRITICAL FIX 1: Change max_length to 512 (match paper)
model_name = "microsoft/unixcoder-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples["input"],
        truncation=True,
        max_length=512,  # CHANGED FROM 128 to 512
        padding=False    # Use dynamic padding
    )

#tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=[col for col in dataset["train"].column_names if col not in ["output"]]
)
tokenized_datasets = tokenized_datasets.rename_column("output", "labels")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# CRITICAL FIX 2: Fix metrics for binary classification
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = evaluate.load("accuracy").compute(predictions=predictions, references=labels)["accuracy"]
    # CHANGED: Use 'binary' instead of 'macro' for binary classification
    precision = evaluate.load("precision").compute(predictions=predictions, references=labels, average="binary")["precision"]
    recall = evaluate.load("recall").compute(predictions=predictions, references=labels, average="binary")["recall"]
    f1 = evaluate.load("f1").compute(predictions=predictions, references=labels, average="binary")["f1"]

    # Calculate FPR
    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "fpr": fpr}

# CRITICAL FIX 3: Better training setup
num_labels = len(set(dataset["train"]["output"]))
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    learning_rate=2e-5,              # Match paper
    per_device_train_batch_size=16,  # Larger if GPU allows
    per_device_eval_batch_size=32,
    num_train_epochs=5,              # CHANGED: More epochs
    weight_decay=0.01,
    logging_steps=50,
    warmup_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train and evaluate
trainer.train()

"""
# CRITICAL FIX 4: Actually evaluate on test set!
test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
print("\n" + "="*50)
print("FINAL TEST RESULTS")
print("="*50)
for key, value in test_results.items():
    if key.startswith('eval_'):
        print(f"{key.replace('eval_', '').upper()}: {value:.4f}")

print(f"\nTarget from paper (UniXcoder/CodeBERT on Devign): F1~0.61")
print(f"Your result: F1={test_results['eval_f1']:.3f}")
"""


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
Upload devign files


Saving devign_0-512_validate.json to devign_0-512_validate.json
Saving devign_0-512_train.json to devign_0-512_train.json
Saving devign_0-512_test.json to devign_0-512_test.json


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shama-aus (shama-aus-nit-kurukshetra) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Fpr
1,0.600300,0.573206,0.678980,0.685413,0.471014,0.558339,0.163620
2,0.524700,0.556222,0.708117,0.704441,0.555556,0.621202,0.176417
3,0.459900,0.651686,0.701873,0.740113,0.474638,0.578366,0.126143
4,0.318800,0.779111,0.691467,0.631874,0.679952,0.655032,0.299817


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Fpr
1,0.600300,0.573206,0.678980,0.685413,0.471014,0.558339,0.163620
2,0.524700,0.556222,0.708117,0.704441,0.555556,0.621202,0.176417
3,0.459900,0.651686,0.701873,0.740113,0.474638,0.578366,0.126143
4,0.318800,0.779111,0.691467,0.631874,0.679952,0.655032,0.299817
5,0.227200,0.912540,0.693028,0.656992,0.601449,0.627995,0.237660


'\n# CRITICAL FIX 4: Actually evaluate on test set!\ntest_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])\nprint("\n" + "="*50)\nprint("FINAL TEST RESULTS")\nprint("="*50)\nfor key, value in test_results.items():\n    if key.startswith(\'eval_\'):\n        print(f"{key.replace(\'eval_\', \'\').upper()}: {value:.4f}")\n\nprint(f"\nTarget from paper (UniXcoder/CodeBERT on Devign): F1~0.61")\nprint(f"Your result: F1={test_results[\'eval_f1\']:.3f}")\n'